In [1]:
# Decision Tree

In [2]:
# load data base
import numpy as np
import pandas as pd

In [3]:
train_df = pd.read_csv('data/train.csv')

In [4]:
train_df.head()

,behavior_sexualRisk,behavior_eating,behavior_personalHygine,intention_aggregation,intention_commitment,attitude_consistency,attitude_spontaneity,norm_significantPerson,norm_fulfillment,perception_vulnerability,perception_severity,motivation_strength,motivation_willingness,socialSupport_emotionality,socialSupport_appreciation,socialSupport_instrumental,empowerment_knowledge,empowerment_abilities,empowerment_desires,ca_cervix
0,10,15,15,10,15,9,10,5,11,15,10,15,15,15,10,15,15,15,15,0
1,10,11,14,10,15,10,10,5,15,14,10,15,9,9,4,3,14,11,15,0
2,10,15,14,10,11,10,8,5,11,15,10,15,15,15,10,15,15,15,15,0
3,10,14,11,10,15,9,10,5,15,15,10,15,13,6,6,12,15,11,14,0
4,10,15,15,6,11,7,6,5,11,13,10,15,15,11,10,15,11,11,15,0


In [5]:
train_df.shape

(58, 20)

# Algoritmo

In [6]:
ejemplos :pd.DataFrame = train_df
atributos : list = train_df.columns

In [7]:
def calc_total_entropy(train_data: pd.DataFrame, label: str, class_list: list)-> float:
    # train_data the entire dataset
    # label the name of the class column
    # class_list list of values that takes label
    total_row = train_data.shape[0]
    total_entropy = 0

    for c in class_list:
        total_class_count = train_data(train_data[label] == c).shape[0] #number of the class
        total_class_entropy = -(total_class_count/total_row)*np.log2(total_class_count/total_row) # entropy of entire class
        total_entropy += total_class_entropy
    return total_entropy

In [8]:
def calc_entropy(feature_value_data: pd.DataFrame, label, class_list: list)-> float:
    # feature has the especific value of a feature (column)
    # label the name of the class
    # class_list different values that takes the class
    class_count = feature_value_data.shape[0]
    entropy = 0
    for c in class_list:
        label_class_count = feature_value_data[feature_value_data[label] == c].shape[0]
        entropy_class = 0
        if label_class_count != 0:
            probability_class = label_class_count/class_count
            entropy_class = - probability_class*np.log2(probability_class)
        entropy += entropy_class
    return entropy

In [9]:
def calc_info_gain(feature_name, train_data, label, class_list):
    feature_value_list = train_data[feature_name].unique() # unique values of the feature
    total_row = train_data.shape[0]
    feature_info = 0.0

    for feature_value in feature_value_list:
        feature_value_data = train_data[ train_data[feature_name] == feature_value ]
        feature_value_count = feature_value_data.shape[0]
        feature_value_entropy = calc_entropy(feature_value_data, label, class_list)
        feature_value_probability = feature_value_count/total_row
        feature_info += feature_value_probability*feature_value_entropy # diapo 31

    return calc_total_entropy(train_data, label, class_list) - feature_info    # diapo 42

In [16]:
def find_most_informative_feature(train_data, label, class_list):
    features_list = train_data.columns.drop(label)
    max_info_gain = -1
    max_info_feature = None

    for feature in features_list:
        feature_info_gain = calc_info_gain(feature, train_data, label, class_list)
        if max_info_gain < feature_info_gain:
            max_info_gain = feature_info_gain
            max_info_feature = feature
            
    return max_info_feature

In [11]:
def generate_sub_tree(feature_name, train_data, label, class_list):
    feature_value_count_dict = train_data[feature_name].value_counts(sort=False) #dictionary of the count of unique feature value
    tree = {} #sub tree or node
    
    for feature_value, count in feature_value_count_dict.iteritems():
        feature_value_data = train_data[train_data[feature_name] == feature_value] #dataset with only feature_name = feature_value
        
        assigned_to_node = False #flag for tracking feature_value is pure class or not
        for c in class_list: #for each class
            class_count = feature_value_data[feature_value_data[label] == c].shape[0] #count of class c

            if class_count == count: #count of (feature_value = count) of class (pure class)
                tree[feature_value] = c #adding node to the tree
                train_data = train_data[train_data[feature_name] != feature_value] #removing rows with feature_value
                assigned_to_node = True
        if not assigned_to_node: #not pure class
            tree[feature_value] = "?" #as feature_value is not a pure class, it should be expanded further, 
                                      #so the branch is marking with ?
            
    return tree, train_data

In [12]:
def make_tree(root, prev_feature_value, train_data, label, class_list):
    if train_data.shape[0] != 0: #if dataset becomes enpty after updating
        max_info_feature = find_most_informative_feature(train_data, label, class_list) #most informative feature
        tree, train_data = generate_sub_tree(max_info_feature, train_data, label, class_list) #getting tree node and updated dataset
        next_root = None
        
        if prev_feature_value != None: #add to intermediate node of the tree
            root[prev_feature_value] = dict()
            root[prev_feature_value][max_info_feature] = tree
            next_root = root[prev_feature_value][max_info_feature]
        else: #add to root of the tree
            root[max_info_feature] = tree
            next_root = root[max_info_feature]
        
        for node, branch in list(next_root.items()): #iterating the tree node
            if branch == "?": #if it is expandable
                feature_value_data = train_data[train_data[max_info_feature] == node] #using the updated dataset
                make_tree(next_root, node, feature_value_data, label, class_list) #recursive call with updated datasetj

In [13]:
def id3(train_data_m, label):
    train_data = train_data_m.copy() #getting a copy of the dataset
    tree = {} #tree which will be updated
    class_list = train_data[label].unique() #getting unqiue classes of the label
    make_tree(tree, None, train_data, label, class_list) #start calling recursion
    return tree

In [14]:
train_df.columns[-1]

'ca_cervix'

In [17]:
tree = id3(train_df, 'ca_cervix')

TypeError: 'DataFrame' object is not callable